# Baseline de algoritmos de xtracción de palabras clave

## Algoritmos básicos: TextRank y Fractalidad

### Importes para librerías

In [22]:
from math import *
from math import sqrt
import string
# import csv
import operator
# import random
import pandas as pd
#librerias necesarias para text rank
from collections import OrderedDict
import numpy as np
import spacy
#import nlp

#Listado de STOPWORDS dependiendo del lenguaje
#from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.es.stop_words import STOP_WORDS

In [23]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 6.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [24]:
nlp = spacy.load('es_core_news_sm')

## Definición de la clase TextRank

In [25]:
class TextRank4Keyword():
    """Extract keywords from text"""

    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 100 # iteration steps
        self.node_weight = None # save keywords and its weight


    def set_stopwords(self, stopwords):
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences

    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab

    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs

    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())

    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1

        # Get Symmeric matrix
        g = self.symmetrize(g)

        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm

        return g_norm


    def get_keywords(self, number=10):
        """Print top number keywords"""
        keysw={}
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            keysw[key] =value
            if i > number:
                break
        return keysw


    def analyze(self, text,
                candidate_pos=['NOUN', 'PROPN'],
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""

        # Set stop words
        self.set_stopwords(stopwords)

        # Pare text by spaCy
        doc = nlp(text)

        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words

        # Build vocabulary
        vocab = self.get_vocab(sentences)

        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)

        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]

        self.node_weight = node_weight


## Definición de la función Grado de Fractalidad

In [26]:
#solamente se calcula el grado de fractalidad de las palabras que tengan mas de uno de frecuencia
def fractalidad(palabras,vocabulario,frec,dist):
    N=len(palabras)                                     #El número de tokens de todo el texto
    gf={}
    cajas_index=set()
    voc=[]                                             #la variable voc contendra cada sintagma con frecuencia mayor que 1, por que las otras palabras tendrán 0 de grado de fractaldiad
    for p in vocabulario:                              #Esto se puede hacer fuera del algoritmo, pero se incluye para evitar ese calculo innecesario
        if(p not in voc):
            if(frec[p]>1):
                if(p not in STOP_WORDS):
                    if(len(p)>1):
                        voc.append(p)
    # print("Text size: ",N)
    # print("Vocabulary: ",len(voc))
    for p in voc:
        rcajas=dist[p]
        M=frec[p]
        dfw=0.0
        nsh=0.0
        for s in range(1,N+1):
            noc=0
            for e in rcajas:
                cajas_index.add(ceil(int(e)/s))
            noc=len(cajas_index)
            cajas_index.clear()
            ns=N/s
            if(M<=ns):
                nsh=M
            else:
                nsh=M/(1+(M-1)/(N-1)*(s-1))
            dfw=dfw+fabs(log(nsh/noc))
        gf[p]=dfw
    return gf    #regresamos un diccionario

In [27]:
def distribucion(palabras,vocabulario):
    N=len(palabras)
    ncajas=[]
    cajas={}
    frecuencias={}
    for p in vocabulario:
        ncajas.clear()
        i=0
        M=palabras.count(p)
        while(i<N):
            if(p == palabras[i]):
                ncajas.append(i+1)
            i=i+1
        frecuencias[p]=M
        cajas[p]=ncajas[:]
    return frecuencias,cajas

## Lectura de archivo de entrada

In [28]:
#Lectura de archivo para generación de vocabulario
def cargar_datos(filename):
    textopos = []
    textoneg =[]
    with open(filename, encoding="utf-8-sig") as f:
        texto = pd.read_csv(f)
        for _, r in texto.iterrows():
            if r['ds0':'ds3'].sum(axis=0) == 1:
                textoneg.append(r['Texto'])
            else:
                textopos.append(r['Texto'])
        for row in textopos:
            #Pasar a minusculas
            row = ' '.join(row)
            row = row.lower()
    #Eliminar puntuación
        for row in textoneg:
            row = ' '.join(row)
            row = row.lower()
    textopos =' '.join(textopos)
    textoneg=' '.join(textoneg)
    textopos = textopos.translate(str.maketrans('', '', string.punctuation))
    textoneg = textoneg.translate(str.maketrans('', '', string.punctuation))
    textopos = textopos.translate(str.maketrans('', '', '¿?¡!.:,;—“”0123456789’'))
    textoneg = textoneg.translate(str.maketrans('', '', '¿?¡!.:,;—“”0123456789’'))
    palabraspos = textopos.split()
    palabrasneg = textoneg.split()
    textop=""
    texton=""
    #rearmamos el texto debido a que existen carácteres especiales
    for w in palabraspos:
        textop=textop+w+' '
    for w in palabrasneg:
        texton=texton+w+' '
    return textop, texton

### Lectura de documentos

In [29]:
def lee_documento(filename='NULL',texto=''):
    if filename != 'NULL':
        textop, texton=cargar_datos(filename)
    textop = textop.lower()
    texton = texton.lower()
    #obtenemos el vocabulario
    tokensp=textop.split()
    vocabulariop=[]
    vocabularion=[]
    tokensn = texton.split()
    for t in tokensp:
        if(t not in vocabulariop):
            vocabulariop.append(t)
    for t in tokensn:
        if(t not in vocabularion):
            vocabularion.append(t)
    #variables de procesamiento
    distp={}
    distn={}
    frecn={}
    frecp={}
    frecp,distp=distribucion(tokensp,vocabulariop)
    frecn, distn = distribucion(tokensn, vocabularion)
    return frecp,distp,tokensp,vocabulariop,textop,frecn,distn,tokensn,vocabularion,texton

## Ejecución de algoritmos

In [30]:
#Lectura de documento para el corpus de suicidio
frecp,distp,tokensp,vocabulariop,textop,frecn,distn,tokensn,vocabularion,texton = lee_documento('/home/m/MGP/maestria/proyecto_tesis/baseline/Datasets/SMS_DATA_ORIGINAL.csv')

In [10]:
print('Frecuencias:', len(frecp))
print("Distribuciones: ", len(distp))
print("Tokens: ", len(tokensp))
print('Vocabulario: ', len(vocabulariop))
print('Texto: ', len(textop))


Frecuencias: 9737
Distribuciones:  9737
Tokens:  147703
Vocabulario:  9737
Texto:  786991


In [11]:
textop

'he hecho lo que toda persona puede hacer que estoy en mi deber y derecho ya que todo lo que me sucede son calumnias y complot hacia mí e iré a más las mentiras no las soporto no me gustan las contiendas personas inventan rumorean calumnias me persiguen pisotean mi imagen se burlan de mí sufro insomnio tengo mareos y ansiedad y se me cae el pelo y tengo sensación de vómito hago arcadas estoy triste lloro porque no me valgo sola no tengo iniciativa para realizar cosas como hacer la limpieza en casa y guisar este mes de agosto en concreto el pasado día tuvimos un incidente con la hermana mayor de mi pareja estando de vacaciones en la casa que tienen mis suegros en la playa se comportó agresiva y verbalmente con mi pareja y conmigo se tuvo que llamar veces a la policía local tuvieron que desplazarse hasta alicante desde segovia mis suegros también para intentar solucionar el problema dicho incidente me ha provocado un temor horrible hacia esa persona tengo la sensación que al problema de 

In [12]:
print('Frecuencias:', len(frecn))
print("Distribuciones: ", len(distn))
print("Tokens: ", len(tokensn))
print('Vocabulario: ', len(vocabularion))
print('Texto: ', len(texton))

Frecuencias: 7377
Distribuciones:  7377
Tokens:  98983
Vocabulario:  7377
Texto:  515324


In [13]:
texton

'mi estado emocional es sumamente inestable debido al estrés que supone tanto la falta de contacto con mis hijos como la impotencia que siento al constatar que los especialistas que han seguido el caso de mis hijos no han querido o no han sabido ver la manipulación que ejerce su padre sobre ellos estoy de vacaciones siento preocupación por análisis de sangre con transaminasas muy altas hice una repetición urgente de la analítica los valores de las transaminasas están bien me siento como quedarme en blanco y no estar pensado en nada me siento como si yo no valiera para nada estoy muy nerviosa tengo noches de pesadillas y calambres siento ganas de comer mucho y otros días de no comer nada siento no tener retentiva o memoria estoy agobiada y no tengo estabilización ni equilibrio estoy muy nerviosa y mal humorada me siento agobiada y con inestabilidad paso en enfados y discusiones en el trabajo con compañeras que por dimesydirétes cuestionan mi trabajo y dicen que tomo bebida de licor esto

### Grado de Fractalidad

In [14]:
#ejecución de algoritmo Grado de Fractalidad
def grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw=False,regresa_df=True,top_n=np.inf,escribe_arch=False):
    frac_x=fractalidad(tokens,vocabulario,frec,dist)
    sorted_x = sorted(frac_x.items(), key=operator.itemgetter(1), reverse=True)
    # print('Time GF: '+str(elapsed_time))

    #Imprimir y guardar resultados de GF
    if regresa_df:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            df = [dato[0] for dato in df]
            by_MC=pd.DataFrame(df, columns=['word'])
        else:
            by_MC=pd.DataFrame(df, columns=['word','frecuency','Degree_of_fractality','Combined_measure'])
        if escribe_arch:
            by_MC.to_csv('GF.csv')
    else:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            by_MC = [dato[0] for dato in df]
        else:
            by_MC = df
        if escribe_arch:
            print('\nNo se tiene implementada la escritura de archivo cuando regresa_df==False\n')
    return by_MC

def use_gf(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tokens=texto.split()
    vocabulario=[]
    for t in tokens:
        if(t not in vocabulario):
            vocabulario.append(t)
    #variables de procesamiento
    dist={}
    frec={}
    frec,dist=distribucion(tokens,vocabulario)
    df = grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw,regresa_df,top_n,escribe_arch)
    return df

fractPositivas = use_gf(textop,regresa_kw=True,regresa_df=True,top_n=57)
fractNegativas = use_gf(texton,regresa_kw=True,regresa_df=True,top_n=57)

In [15]:
fractPositivas

,word,frecuency,Degree_of_fractality,Combined_measure
0,caca,23,125172.298308,170450.602904
1,recogido,13,117297.911710,130663.228989
2,epigastrio,13,114829.319993,127913.357656
3,rituales,12,116211.203602,125412.951508
4,ok,12,115319.796097,124450.961246
5,observaciones,19,96055.374999,122831.156670
6,capsulas,11,115071.236657,119834.344127
7,calzoncillo,11,113917.125279,118632.460980
8,conjuntivitis,9,109502.900295,104492.322368
9,oftalmóloga,8,107430.311784,97019.238871


In [16]:
fractNegativas

,word,frecuency,Degree_of_fractality,Combined_measure
0,lavadora,33,60811.061206,92342.444141
1,ansium,28,60280.895388,87235.981897
2,tendido,22,59131.211519,79379.079488
3,fregado,17,63527.341626,78167.148981
4,tripa,17,57335.809170,70548.784550
5,movido,8,73046.327163,65967.406647
6,auriculares,9,68716.020363,65571.747710
7,naltrexona,8,70093.963440,63301.156531
8,dolorido,11,60743.506400,63257.843236
9,hemicraneal,13,55242.870076,61537.427883


In [17]:
#Guarda los resultados del grado de fractalidad en un CSV. Siempre hay que cambiar la versión y comentar para no sobreescribir
# fractPositivas.to_csv('/home/m/MGP/maestria/proyecto_tesis/baseline/output/fractalidad_positivas_resultados_prueba2.csv')
# fractNegativas.to_csv('/home/m/MGP/maestria/proyecto_tesis/baseline/output/fractalidad_negativas_resultados_prueba2.csv')

### TextRank

In [31]:
#Cambio del máximo de tamaño de texto para que quepa el corpus (1.3 millones)
nlp.max_length = 1500000

In [32]:
#ejecución de algoritmo de TextRank
# start_time = time()
def use_TextRank(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tr4w = TextRank4Keyword()
    tr4w.analyze(texto, candidate_pos = ['NOUN','PROPN'], window_size=4, lower=False)
    kwTR=tr4w.get_keywords(100)

    #Guardar resultados de TextRank
    if regresa_df:
        if top_n!=np.inf:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())[:top_n]
                dftr=pd.DataFrame(salida, columns=['word'])
        else:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())
                dftr=pd.DataFrame(salida, columns=['word'])
    else:
        if top_n!=np.inf:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
            else:
                dftr = list(kwTR.keys())[:top_n]
        else:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()]
            else:
                dftr = list(kwTR.keys())
        # elapsed_time = time() - start_time
        # print('Time TextRank: '+str(elapsed_time))
        if escribe_arch:
            dftr.to_csv('TextRank.csv')
    return dftr

trpos = use_TextRank(textop,top_n=57)
trneg = use_TextRank(texton,top_n=57)

In [33]:
trpos

['trabajo',
 'cosas',
 'casa',
 'semana',
 'tiempo',
 'vida',
 'ansiedad',
 'ganas',
 'horas',
 'madre',
 'noche',
 'mañana',
 'cabeza',
 'problemas',
 'familia',
 'situación',
 'sueño',
 'miedo',
 'medicación',
 'momento',
 'sensación',
 'problema',
 'dolor',
 'ánimo',
 'tratamiento',
 'gente',
 'pareja',
 'años',
 'hora',
 'marido',
 'mujer',
 'padre',
 'gracias',
 'persona',
 'general',
 'cosa',
 'doctora',
 'momentos',
 'tema',
 'tensión',
 'hija',
 'comida',
 'hijo',
 'hijos',
 'mes',
 'vacaciones',
 'consulta',
 'hermana',
 'cambio',
 'intento',
 'médico',
 'cuestionario',
 'pastillas',
 'cambios',
 'forma',
 'pensamientos',
 'semanas']

In [34]:
trneg

['vida',
 'casa',
 'cosas',
 'dolor',
 'madre',
 'cabeza',
 'ansiedad',
 'ganas',
 'tiempo',
 'gracias',
 'semana',
 'trabajo',
 'problemas',
 'noche',
 'perro',
 'medicación',
 'años',
 'persona',
 'mañana',
 'horas',
 'comida',
 'gente',
 'miedo',
 'cosa',
 'familia',
 'mierda',
 'sensación',
 'sueño',
 'padre',
 'cama',
 'noches',
 'hija',
 'mundo',
 'dolores',
 'momento',
 'cuerpo',
 'hora',
 'ánimo',
 'situación',
 'médico',
 'problema',
 'momentos',
 'tensión',
 'daño',
 'pensamientos',
 'hospital',
 'intento',
 'marido',
 'dinero',
 'pena',
 'nervios',
 'personas',
 'tratamiento',
 'sentido',
 'hijos',
 'pastillas',
 'calle']

In [35]:
#Guarda los resultados del TextRank en un txt. Siempre hay que cambiar la versión y comentar para no sobreescribir
# with open('/home/m/MGP/maestria/proyecto_tesis/baseline/output/TextRank_resultados_positivos_prueba2.txt','w') as output:
#     output.write(str(trpos))
# with open('/home/m/MGP/maestria/proyecto_tesis/baseline/output/TextRank_resultados_negativos_prueba2.txt','w') as output:
#     output.write(str(trneg))

## Comparación de los resultados de fractalidad y TextRank con el conjunto de Palabras Prototípicas

In [36]:
with open('/home/m/MGP/maestria/proyecto_tesis/baseline/Palabras_proto/palabras_suicidio_esp.csv', encoding="utf-8-sig") as f:
        palabrasProto = pd.read_csv(f)
        

In [37]:
palabrasProto

,Unnamed: 0,Palabra,Positiva o Negativa,pos-neg
0,0,bien,Positiva,1
1,1,nerviosa,Negativa,-1
2,2,mal,Negativa,-1
3,3,contenta,Positiva,1
4,4,difícil,Negativa,-1
5,5,tranquila,Positiva,1
6,6,capaz,Positiva,1
7,7,inútil,Negativa,-1
8,8,triste,Negativa,-1
9,9,sola,Negativa,-1


### Función para crear la medida de Jaccard

In [38]:
#Medida Jaccard que mide la proporción de elementos de un conjunto en otro
def jaccard(s1: set, s2: set):
    return len(s1 & s2) / len(s1 | s2)   

In [39]:
# jaccFractalidadPos = jaccard(set(palabrasProto['Palabra']), set(fractPositivas['word']))
jaccTRPos = jaccard(set(palabrasProto['Palabra']), set(trpos))
# jaccFractalidadNeg = jaccard(set(palabrasProto['Palabra']), set(fractNegativas['word']))
jaccTRNeg = jaccard(set(palabrasProto['Palabra']), set(trneg))
# _, conteosTR = jaccard(texto, dftr)
# _, conteosFract= jaccard(texto, list(df1['word']))
# _, conteosProto= jaccard(texto, list(palabrasProto['Palabra']))

# print('Jaccard de fractalidad positivas: ', jaccFractalidadPos)

print('Jaccard de TextRank positivas: ', jaccTRPos)

# print('Jaccard de fractalidad negativas: ', jaccFractalidadNeg)

print('Jaccard de TextRank negativas: ', jaccTRNeg)


Jaccard de TextRank positivas:  0.017857142857142856
Jaccard de TextRank negativas:  0.017857142857142856


In [40]:
#Función para obtener las palabras coinsidentes:
def coinsidencia(conjuntoA, conjuntoB):
    matches = []
    for palabra in conjuntoA:
        if palabra in conjuntoB:
            matches.append(palabra)
    conteo = len(matches)
    return matches, conteo

In [41]:
coinTRPos, conteosTRpos = coinsidencia(list(palabrasProto['Palabra']), trpos)
coinTRNeg, conteosTRNeg = coinsidencia(list(palabrasProto['Palabra']), trneg)

In [42]:
print(coinTRPos)
print(conteosTRpos)
print(coinTRNeg)
print(conteosTRNeg)

['vida', 'noche']
2
['vida', 'noche']
2
